## Created by <a href="https://github.com/yunsuxiaozi">yunsuxiaozi</a> 2024/7/4

#### 昨天在手机上刷视频的时候,看到了这个视频:<a href="https://www.bilibili.com/video/BV1da4y1k78p/?spm_id_from=333.337.search-card.all.click">当我开发出史料检索RAG应用，正史怪又该如何应对？</a>里面提到的RAG技术是我非常感兴趣的,所以我就手动实现了一个非常简单的RAG检索的程序。

#### 这里使用的大语言模型是<a href="https://www.kaggle.com/models/thudm-keg/chatglm2/PyTorch/6b/1">chatglm2</a>,这里使用的数据集是<a href="https://www.kaggle.com/datasets/terrychanorg/classifypet">中文宠物文章分类</a>,选择这个数据集没什么理由,因为我想把自己的重点放在学技术上,不想花太多时间找数据集。

#### 注:由于我只是RAG的初学者,所以只实现了RAG的大致流程,对于具体的技术细节也都采用了最简单的方法实现。同时,对于RAG的理解可能也存在一些误解,如果读者发现错误可以与我沟通并指正我的错误。



## 1.安装必要的python库

#### 一个是对向量进行相似性比较的库,一个是大语言模型使用的transformers库

In [1]:
#-q是安装的时候静默,减少输出日志信息
!pip install -q faiss-cpu==1.7.3 #Facebook AI Similarity Search,由Facebook AI Research团队开发的库，用于高效相似性搜索和密集向量聚类
!pip install -q transformers==4.33.0  #安装transformers

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kaggle-environments 1.14.11 requires transformers>=4.33.1, but you have transformers 4.33.0 which is incompatible.


## 2.导入数据集。

#### 首先读取数据集,然后把文本按照句子来划分,简单去了几个停用词,根据句子长度筛选一些有价值的句子。句子太短的话可能连问题的长度都达不到。更别说回答问题了。为了节约时间,这里选择了前100句话来做个示范。

In [2]:
import numpy as np#进行矩阵运算的库
import json#用于读取和写入json数据格式
import re#用于正则表达式提取

print(f"read_data")
train=[]
with open("/kaggle/input/classifypet/train.json",'r',encoding='utf-8') as f:
    for line in f:
        train.append(json.loads(line))
train=[train[i]['sentence'] for i in range(len(train)) if train[i]['label']==1]
print(f"len(train):{len(train)}")

texts=[]
for sentence in train:
    s_texts=re.split('\\。|\\？|\\！|\n',sentence)#sentence.split('\n')
    stopwords=['你','我','他','她','我们']
    for i in range(len(s_texts)):
        for stop_word in stopwords:
            s_texts[i]=re.sub(stop_word, '', s_texts[i])
    texts+=s_texts
#文本太短的话就没什么用了
texts=[text for text in texts if len(text)>7]#今天天气真好啊
print(f"len(texts):{len(texts)}")
texts=texts[:100]
print(f"texts[0]:{texts[0]}")

read_data
len(train):1530
len(texts):42189
texts[0]:柯基犬的性格特点，喂养柯基犬要注意哪些方面


## 3.导入大语言模型。


#### 这里导入了大语言模型和分词器。

In [3]:
from transformers import AutoTokenizer, AutoModel#自动加载分词器和自动加载预训练模型

llm_tokenizer = AutoTokenizer.from_pretrained("/kaggle/input/chatglm2/pytorch/6b/1", trust_remote_code=True)
#允许远程加载模型,模型加载的时候将模型转成半精度浮点数并移动到GPU上。
llm_model = AutoModel.from_pretrained("/kaggle/input/chatglm2/pytorch/6b/1", trust_remote_code=True).half().cuda()
#将模型调成推理模式。
llm_model.eval()#llm_model = 

2024-07-04 03:03:15.912794: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-04 03:03:15.912896: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-04 03:03:16.041627: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

ChatGLMForConditionalGeneration(
  (transformer): ChatGLMModel(
    (embedding): Embedding(
      (word_embeddings): Embedding(65024, 4096)
    )
    (rotary_pos_emb): RotaryEmbedding()
    (encoder): GLMTransformer(
      (layers): ModuleList(
        (0-27): 28 x GLMBlock(
          (input_layernorm): RMSNorm()
          (self_attention): SelfAttention(
            (query_key_value): Linear(in_features=4096, out_features=4608, bias=True)
            (core_attention): CoreAttention(
              (attention_dropout): Dropout(p=0.0, inplace=False)
            )
            (dense): Linear(in_features=4096, out_features=4096, bias=False)
          )
          (post_attention_layernorm): RMSNorm()
          (mlp): MLP(
            (dense_h_to_4h): Linear(in_features=4096, out_features=27392, bias=False)
            (dense_4h_to_h): Linear(in_features=13696, out_features=4096, bias=False)
          )
        )
      )
      (final_layernorm): RMSNorm()
    )
    (output_layer): Linear(in_

#### 为了观察RAG前和RAG后模型的输出结果的区别,我们这里在RAG前测试一下模型。

In [4]:
question='用户问题:请问柯基犬有几种?'
output,history = llm_model.chat(llm_tokenizer,question,max_length=2048, history=[])
print(f"output:{output}")

output:柯基犬是一种小型短腿犬，常见的品种有柯基犬、艾比奴犬、卡宾犬等。但是，柯基犬不是一个品种，而是一个犬种，因此没有柯基犬这种犬种。


## 4.数据的embedding和question的embedding


#### 这里选择的是"BAAI/bge-large-zh-v1.5",它是对文本进行embedding的模型,它也有对应的分词器.

#### 这里的操作就是先对文本分词,传入embedding的model得到输出model_output,输出的last_hidden_state是(batch_size,max_seq_len,embed_dim)。由于一个句子传入模型之后会变成[CLS] + 句子 + [SEP],据说cls里包含了文本的主要信息,所以进行了CLS_pool ,也就是把(batch_size,max_seq_len,embed_dim)变成了(batch_size,embed_dim),也就是代码里的model_output['last_hidden_state'][:, 0],然后对向量进行了batch的归一化操作。



In [5]:
import torch#深度学习框架pytorch
#允许远程加载模型,模型加载的时候将模型转成半精度浮点数并移动到GPU上。
embed_tokenizer = AutoTokenizer.from_pretrained("BAAI/bge-large-zh-v1.5", trust_remote_code=True)
embed_model=AutoModel.from_pretrained("BAAI/bge-large-zh-v1.5",trust_remote_code=True)
#模型转换成评估模式
embed_model.eval()
def embedding(texts,batch_size=10):
    embed_vector=[]
    for i in range(0,len(texts),batch_size):
        text_input=embed_tokenizer(texts[i:i+batch_size], max_length=512, padding=True, truncation=True,
                                               return_tensors='pt')
        with torch.no_grad():
            model_output = embed_model(**text_input)
        #,截取开头的cls,据说cls有了全局的特征,所以[:, 0]就是cls pool.
        batch_embeddings=model_output['last_hidden_state'][:, 0]
        #(batch_size,1024)
        batch_embeddings=torch.nn.functional.normalize(batch_embeddings, p=2, dim=1)
        embed_vector.append(batch_embeddings.numpy())
    return np.vstack(embed_vector)

knowledge=embedding(texts)
question=['请问柯基犬有几种?']
question=embedding(question)

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/394 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/439k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.30G [00:00<?, ?B/s]

## 5.向量相似性搜索

In [6]:
import faiss#Facebook AI Similarity Search,由Facebook AI Research团队开发的库，用于高效相似性搜索和密集向量聚类 
#创建了基于L2距离的平面索引
vector = faiss.IndexFlatL2(knowledge.shape[1])
#将向量加入进去
vector.add(knowledge)                  
topk=3
#找到和问题最相似的topk的结果的index和distance
distance,index = vector.search(question, topk)
print(f"index:{index},distance:{distance}")
index=index[0]
for idx in index:
    print(f"text:{texts[idx]}")

index:[[ 5 36 47]],distance:[[0.646539   0.6598448  0.74260664]]
text:柯基犬及威尔士柯基犬（著名犬种）沃尔士柯基犬，共分两种，卡迪根威尔士柯基犬和彭布罗克威尔士柯基犬
text:柯基犬的毛色一般有三种以上，们的背部颜色主要是黄色或者是黑色
text:很简单，就是们用手抓一把它们的毛，然后立马松开，仔细观察柯基犬的毛会不会在短时间内恢复原状，如果可以，就可以基本判断这是一只纯种柯基犬了


## 6.RAG之后的大语言模型问答

In [7]:
question="请根据以下文本回答用户问题:\n"
for i in range(len(index)):
    question+=f"文本{i+1}:{texts[index[i]]}\n"
question+='用户问题:请问柯基犬有几种?'
print(f"question:{question}")
output,history = llm_model.chat(llm_tokenizer,question,max_length=2048, history=[])
print(f"output:{output}")

question:请根据以下文本回答用户问题:
文本1:柯基犬及威尔士柯基犬（著名犬种）沃尔士柯基犬，共分两种，卡迪根威尔士柯基犬和彭布罗克威尔士柯基犬
文本2:柯基犬的毛色一般有三种以上，们的背部颜色主要是黄色或者是黑色
文本3:很简单，就是们用手抓一把它们的毛，然后立马松开，仔细观察柯基犬的毛会不会在短时间内恢复原状，如果可以，就可以基本判断这是一只纯种柯基犬了
用户问题:请问柯基犬有几种?
output:根据文本1，柯基犬分为两种，卡迪根威尔士柯基犬和彭布罗克威尔士柯基犬。因此，柯基犬有两种。
